In [ ]:
import pandas as pd
import requests
import os
import cv2
import numpy as np

In [ ]:
# Create a directory to save the images if it doesn't already exist
output_folder = 'amazon_test_images'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Load the dataset into a pandas DataFrame
df = pd.read_csv('Downloads/amazon_ml/student_resource 3/dataset/test.csv')

In [ ]:
df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [ ]:
full_set = set(range(131288))
numbers_set = set(df['index'].tolist())
print(len(numbers_set))

missing_numbers = full_set - numbers_set
missing_numbers = sorted(missing_numbers)
print(missing_numbers)

131187
[763, 1787, 1788, 1950, 2871, 3150, 3842, 3864, 3865, 4613, 8480, 10662, 15098, 15540, 15541, 15662, 16144, 16751, 17479, 17480, 18514, 18515, 18752, 19567, 19568, 20238, 20239, 21619, 21620, 21621, 24950, 24951, 25115, 25116, 27580, 31018, 31019, 33803, 33804, 33805, 34057, 34922, 36254, 36255, 39917, 43657, 47185, 54862, 55990, 58364, 58365, 58366, 62330, 63346, 63347, 63792, 63807, 64168, 64848, 66951, 66952, 69367, 69368, 69369, 71253, 73662, 75610, 75611, 75612, 86447, 86448, 88676, 89529, 89530, 101003, 104531, 108082, 108083, 109399, 112480, 119400, 119401, 120077, 120078, 120112, 120113, 120450, 120451, 121198, 121199, 121862, 122084, 124444, 124445, 124692, 124693, 127110, 127615, 128461, 129531, 129532]


In [ ]:
# Function to download and save an image
def download_image(index, url, folder_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for HTTP request errors
        image_path = os.path.join(folder_path, f'{index}.jpg')  # Save as .jpg or .png based on your needs
        with open(image_path, 'wb') as file:
            file.write(response.content)
        print(f'Successfully downloaded {index}')
    except Exception as e:
        print(f'Failed to download {index}: {e}')

In [ ]:
# Loop through the dataset and download images
for index, row in df.iterrows():
  if(index>=131187):
    image_url = row[1]  # URL is in the first column
    download_image(index, image_url, output_folder)

In [ ]:
invalid_links = [520,7754,17319,26576,28993,34979,37141,39111,39962,48366,63572,65462,96679,101790,103787,111183,113130,120294]
#520,7754,17319,26576,28993,34979,37141,39111,39962,48366,63572,65462,96679,101790,103787,111183,113130,120294

In [ ]:
def find_invalid_links(index, url):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Check for HTTP request errors
    except Exception as e:
        print(f'Failed to download {index}: {e}')
        invalid_links.append(index)

In [ ]:
for index, row in df.iterrows():
  if(index>101790):
    image_url = row[0]  # URL is in the first column
    find_invalid_links(index, image_url)

C:\Users\ekamb\AppData\Local\Temp\ipykernel_7364\4081435416.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_url = row[0]  # URL is in the first column


Failed to download 103787: 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/3sSrJnc5R58.jpg
Failed to download 111183: 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/caDEyEaRMCm.jpg
Failed to download 113130: 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/VRs4UBsSHaM.jpg
Failed to download 120294: 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/BEJwJEFSTSp.jpg


In [ ]:
invalid_links

[520,
 7754,
 17319,
 26576,
 28993,
 34979,
 37141,
 39111,
 39962,
 48366,
 63572,
 65462,
 96679,
 101790,
 103787,
 111183,
 113130,
 120294]

In [ ]:
# Directory where the original images are stored
image_dir = "amazon_train_images"
# Directory where the preprocessed images will be saved
preprocessed_dir = "amazon_train_preprocessed_images"
#os.makedirs(preprocessed_dir, exist_ok=True)

In [ ]:
# List all image files
image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]

In [ ]:
# Sort image files by numerical index
# We assume the format is "{index}_{group_id}.jpg"
image_files.sort(key=lambda x: int(x.split('_')[0]))

In [ ]:
def resize_image(image_path, target_size=(1024, 1024)):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    # Calculate the scaling factor while keeping the aspect ratio
    scale = min(target_size[0] / h, target_size[1] / w)

    # Resize the image with better interpolation to avoid blurring text
    resized_img = cv2.resize(img, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_CUBIC)

    # Create a new blank image (filled with white) of the target size
    padded_img = np.ones((target_size[0], target_size[1], 3), dtype=np.uint8) * 255

    # Get the top-left corner for centering the resized image
    x_offset = (target_size[1] - resized_img.shape[1]) // 2
    y_offset = (target_size[0] - resized_img.shape[0]) // 2

    # Place the resized image on the padded image
    padded_img[y_offset:y_offset + resized_img.shape[0], x_offset:x_offset + resized_img.shape[1]] = resized_img

    return padded_img

In [ ]:
image_files_50000 = image_files[49:]
image_files_50000

In [ ]:
# Loop through all images and preprocess them
for filename in image_files:
    # Extract the index from the filename
    index = filename.split('_')[0]
    if(int(index)<50000 or int(index)>100000):
      continue

    # Full path of the image
    image_path = os.path.join(image_dir, filename)

    # Preprocess the image (resize and pad)
    preprocessed_image = resize_image(image_path, target_size=(512, 512))

    # Save the preprocessed image with just the index as the file name
    new_filename = f"{index}.jpg"
    save_path = os.path.join(preprocessed_dir, new_filename)

    cv2.imwrite(save_path, preprocessed_image)
    print(f"Preprocessed & saved: {new_filename}")

Preprocessed & saved: 50000.jpg
Preprocessed & saved: 50001.jpg
Preprocessed & saved: 50002.jpg
Preprocessed & saved: 50003.jpg
Preprocessed & saved: 50004.jpg
Preprocessed & saved: 50005.jpg
Preprocessed & saved: 50006.jpg
Preprocessed & saved: 50007.jpg
Preprocessed & saved: 50008.jpg
Preprocessed & saved: 50009.jpg
Preprocessed & saved: 50010.jpg
Preprocessed & saved: 50011.jpg
Preprocessed & saved: 50012.jpg
Preprocessed & saved: 50013.jpg
Preprocessed & saved: 50014.jpg
Preprocessed & saved: 50015.jpg
Preprocessed & saved: 50016.jpg
Preprocessed & saved: 50017.jpg
Preprocessed & saved: 50018.jpg
Preprocessed & saved: 50019.jpg
Preprocessed & saved: 50020.jpg
Preprocessed & saved: 50021.jpg
Preprocessed & saved: 50022.jpg
Preprocessed & saved: 50023.jpg
Preprocessed & saved: 50024.jpg
Preprocessed & saved: 50025.jpg
Preprocessed & saved: 50026.jpg
Preprocessed & saved: 50027.jpg
Preprocessed & saved: 50028.jpg
Preprocessed & saved: 50029.jpg
Preprocessed & saved: 50030.jpg
Preproce